In [1]:
import networkx as nx
from tqdm import tqdm
from datetime import datetime

In [2]:
start = datetime.now()
file = open('ca-AstroPh.txt','r')
lines = file.readlines()

In [3]:
def dic2list(myDic):
    graph = []
    for i in myDic.keys():
        graph.append(i)
    return graph

In [49]:
nodes = {}
unique_nodes = set()
neighbor_c = {}
for l in lines:
    if '#' in str(l):
        continue
    node = l.replace('\n','').split('\t')
    n1 = int(node[0])
    n2 = int(node[1])
    nodes[(n1,n2)] = 1
    unique_nodes.add(n1)
    if  neighbor_c.get(n1,0) == 0:
             neighbor_c[n1] = []
    neighbor_c[n1].append(n2)
nodes_set = dic2list(nodes)

# len(neighbor_c)
# neighbor_c[4]

[7707, 74614]

In [5]:
def my_bfs(visited, queue, graph, node):
    visited.append(node)
    queue.append(node)
    n_list = []
    while queue:
        s = queue.pop(0)
        n_list.append(s)
        for neighbour in graph[s]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return n_list

In [6]:
def iter_bfs_2(neighbor):
    visited_n = []
    queue_n = []
    n_neighbor = {}
    for a in tqdm(neighbor):
        n_neighbor[a] = my_bfs(queue_n, visited_n,neighbor_c,a)
    return n_neighbor

In [7]:
hc_bfsed =  iter_bfs_2(unique_nodes)

100%|██████████| 18772/18772 [01:11<00:00, 263.49it/s]


In [58]:
len(hc_bfsed[4])
hc_bfsed[4]

[4]

In [8]:
out_sorted = sorted(hc_bfsed, key=lambda k: len(hc_bfsed[k]), reverse=True).copy()

In [9]:
def hill_climbing(hc_s):
    unique_nodes_h = unique_nodes.copy()
    neigh = set()
    for s in hc_s:
        unique_nodes_h.remove(s)
        neigh.add(frozenset(hc_bfsed[s]))
    while  len(hc_s) < 10 :
        temp_dic = {}
        for n in unique_nodes_h:
            temp_dic[n] = neigh.union(hc_bfsed[n])
        selected = sorted(temp_dic, key=lambda k: len(temp_dic[k]), reverse=True)[0]
        neigh.add(frozenset(temp_dic[selected]))
        hc_s.append(selected)
        unique_nodes_h.remove(selected)
    return hc_s
S = []
S.append(out_sorted[0])

hc = hill_climbing(S)
hc

[3, 97939, 68695, 67720, 65252, 33128, 358, 131978, 100471, 885]

### -Lazy Hill Climbing

In [10]:
lhc_bfs = hc_bfsed.copy()
# lhc_bfs_sorted = sorted(lhc_bfs, key=lambda k: len(lhc_bfs[k]), reverse=True)

In [11]:
def lazy_hill_climbing():
    sl = []
    neighbor_sorted_l = out_sorted.copy()
    s1 = neighbor_sorted_l[0]
    sl.append(s1)
    sl_outs = lhc_bfs[s1].copy()

    neighbor_sorted_l.remove(s1)
    for i, n in enumerate(neighbor_sorted_l):
        if len(sl) > 9:
            break
        left = n
        right = neighbor_sorted_l[i + 1]

        mg_l = [a for a in lhc_bfs[left] if a not in sl_outs]

        l = len(mg_l)
        r = len(lhc_bfs[right])

        if l >= r:
            sl.append(n)
            sl_outs.append(lhc_bfs[n])
        else:
            lhc_bfs[n] = mg_l.copy()
            neighbor_sorted_l = sorted(lhc_bfs, key=lambda k: len(lhc_bfs[k]), reverse=True).copy()
    return sl
lhc = lazy_hill_climbing()
lhc

[3, 97939, 68695, 65252, 67720, 33128, 358, 131978, 100471, 885]

In [12]:
print("sorted")
for i in out_sorted[0:10]:
    print(i,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),i  ))))
print("-----------\nhill climbing")
for i in hc:
    print(i,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),i  ))))
print("-----------\nLazy hill climbing")
for i in lhc:
    print(i,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),i  ))))



sorted
3 17903
97939 18
68695 12
65252 10
67720 10
33128 9
358 8
131978 8
100471 8
885 7
-----------
hill climbing
3 17903
97939 18
68695 12
67720 10
65252 10
33128 9
358 8
131978 8
100471 8
885 7
-----------
Lazy hill climbing
3 17903
97939 18
68695 12
65252 10
67720 10
33128 9
358 8
131978 8
100471 8
885 7


In [41]:
print(4,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),4  ))))

4 17903


In [40]:

cost = {v: 1 for v in unique_nodes}

benefit_cost_ratio = {v: len(hc_bfsed[v]) / cst for v, cst in cost.items()}
benefit_cost_ratio[97939]
# values = list(benefit_cost_ratio.values())
# keys = list(benefit_cost_ratio.keys())
# max_index = values.index(max(values))
# selected_node = keys[max_index]
# selected_node

18.0

TypeError: list indices must be integers or slices, not tuple